In [81]:
import logging
import os
#from _helpers import configure_logging

import pypsa
import yaml
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib as plt

from scipy.sparse import csgraph
from itertools import product

from shapely.geometry import Point, LineString
import shapely, shapely.prepared, shapely.wkt

logger = logging.getLogger(__name__)


In [82]:
path = os.path.realpath("networks")+'/base.nc'


In [83]:
n = pypsa.Network(path)

n

INFO:pypsa.io:Imported network base.nc has buses, lines


Network PyPSA-Eur

In [84]:
n.buses

,v_nom,symbol,under_construction,tag_substation,tag_area,lon,lat,country,geometry,type,x,y,carrier,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
name,,,,,,,,,,,,,,,,,,
0,220.0,False,False,False,False,-1.868851,34.786907,algeria,NaN,,0.0,0.0,AC,1.0,0.0,inf,PQ,
1,380.0,False,False,False,False,-0.902003,35.089832,algeria,NaN,,0.0,0.0,AC,1.0,0.0,inf,PQ,
2,220.0,False,False,False,False,0.146211,34.909046,algeria,NaN,,0.0,0.0,AC,1.0,0.0,inf,PQ,
3,220.0,False,False,False,False,3.380172,36.357213,algeria,NaN,,0.0,0.0,AC,1.0,0.0,inf,PQ,
4,220.0,False,False,False,False,0.146611,34.908806,algeria,NaN,,0.0,0.0,AC,1.0,0.0,inf,PQ,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19891,220.0,False,False,False,False,31.011963,-29.813611,south-africa,NaN,,0.0,0.0,AC,1.0,0.0,inf,PQ,
19892,220.0,False,False,False,False,39.277212,-6.317658,tanzania,NaN,,0.0,0.0,AC,1.0,0.0,inf,PQ,
19893,220.0,False,False,False,False,9.976713,37.210589,tunisia,NaN,,0.0,0.0,AC,1.0,0.0,inf,PQ,


In [85]:
pd.set_option('display.max_columns', None)
n.lines


,bus0,bus1,v_nom,num_parallel,length,underground,under_construction,tag_type,tag_frequency,country,geometry,bounds,bus0_lon,bus0_lat,bus1_lon,bus1_lat,bus_0_coors,bus_1_coors,type,x,r,g,b,s_nom,s_nom_extendable,s_nom_min,s_nom_max,s_max_pu,capital_cost,terrain_factor,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
380986751-1,0,9948,220.0,1.0,40.467296,False,False,line,50,algeria,"LINESTRING (-1.8363817 34.78690679999992, -1.8...","MULTIPOINT (-1.8363817 34.78690679999992, -1.8...",-1.868851,34.786907,-1.836382,35.080530,NaN,NaN,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173682341-1,1,9949,380.0,1.0,0.168068,False,False,line,50,algeria,"LINESTRING (-0.9017649 35.08983150000002, -0.9...","MULTIPOINT (-0.9017649 35.08983150000002, -0.9...",-0.902003,35.089832,-0.901765,35.091049,NaN,NaN,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
655093026-1,2,9950,220.0,1.0,0.064988,False,False,line,50,algeria,"LINESTRING (0.1462109 34.90904640000009, 0.146...","MULTIPOINT (0.1462109 34.90904640000009, 0.146...",0.146211,34.909046,0.146611,34.909394,NaN,NaN,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
383315204-1,3,9951,220.0,1.0,98.637025,False,False,line,50,algeria,"LINESTRING (3.3801723 36.71033960000008, 3.383...","MULTIPOINT (3.3801723 36.71033960000008, 3.964...",3.380172,36.357213,3.964184,36.710340,NaN,NaN,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
655093020-1,4,9952,220.0,1.0,0.110799,False,False,line,50,algeria,"LINESTRING (0.1466111 34.90939410000009, 0.146...","MULTIPOINT (0.1466111 34.90939410000009, 0.147...",0.146611,34.908806,0.147301,34.909394,NaN,NaN,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456151088-1,9943,19891,220.0,1.0,0.131901,True,False,cable,50,south-africa,LINESTRING (31.01196310000006 -29.813610599999...,MULTIPOINT (31.01196310000006 -29.813610599999...,31.011290,-29.814250,31.011963,-29.813611,NaN,NaN,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668505779-1,9944,19892,220.0,1.0,36.903154,True,False,cable,50,tanzania,"LINESTRING (39.19949770000008 -6.6373292, 39.1...","MULTIPOINT (39.19949770000008 -6.6373292, 39.2...",39.199498,-6.637329,39.277212,-6.317658,NaN,NaN,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
445315079-1,9945,19893,220.0,1.0,0.948420,True,False,cable,50,tunisia,LINESTRING (9.976712599999923 37.2104494000005...,MULTIPOINT (9.976712599999923 37.2104494000005...,9.968195,37.210449,9.976713,37.210589,NaN,NaN,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
